In [28]:

import pandas as pd 
import calendar
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

df=pd.read_csv('dff_old.csv')

# df.head()

In [29]:
#IMPUTING economic indicators 

xls = pd.ExcelFile('toappend2.xlsx')
print('Sheet names are',xls.sheet_names)

# to read just one sheet to dataframe:
df30_ = pd.read_excel('toappend2.xlsx', sheet_name="daily")
#display(df30_)

df30_["day"] = df30_['Date'].map(lambda x: x.day)
df30_["month"] = df30_['Date'].map(lambda x: x.month)
df30_["month"] = df30_["month"].apply(lambda x: calendar.month_abbr[x].lower())
df30_["year"] = df30_['Date'].map(lambda x: x.year)
df30= df30_.iloc[::-1].reset_index(drop=True)

df['euribor3m'] = 0.0

for i in range(len(df30)):    
    i_raw=df30.iloc[i,:]
    df.loc[(df.day==i_raw.day) & (df.month==i_raw.month) & (df.year==i_raw.year.astype(float)),'euribor3m']=i_raw.Value

df12_ = pd.read_excel('toappend2.xlsx', sheet_name="monthly")
#Harmonised index of consumer prices, Consumer confidence indicator

df12_.columns
df12_.rename(columns={df12_.columns[0]: 'Date', df12_.columns[1]: 'consum_prices_rate',df12_.columns[2]: 'consum_conf_ind'},inplace=True)
df12_=df12_.iloc[3:]

df12_["day"] = df12_['Date'].map(lambda x: x.day)
df12_["month"] = df12_['Date'].map(lambda x: x.month)
df12_["month"] = df12_["month"].apply(lambda x: calendar.month_abbr[x].lower())
df12_["year"] = df12_['Date'].map(lambda x: x.year)
df12= df12_.iloc[::-1].reset_index(drop=True)

#df12.head()
df['consum_prices_rate'] = 0.0
df['consum_conf_ind'] = 0.0

for i in range(len(df12)):    
    i_raw=df12.iloc[i,:]
    #print(i,i_raw.Value)
    df.loc[(df.day<=i_raw.day) & (df.month==i_raw.month) & (df.year==i_raw.year.astype(float)),'consum_prices_rate']=i_raw.consum_prices_rate
    df.loc[(df.day<=i_raw.day) & (df.month==i_raw.month) & (df.year==i_raw.year.astype(float)),'consum_conf_ind']=i_raw.consum_conf_ind    

df4_ = pd.read_excel('toappend2.xlsx', sheet_name="quaretrly")

df4_.columns
df4_.rename(columns={df4_.columns[0]: 'Date', df4_.columns[1]: 'employed',df4_.columns[2]: 'unemployed',df4_.columns[3]: 'unemployed_rate'},inplace=True)
df4_=df4_.iloc[3:]

df4_["month"] = df4_['Date'].map(lambda x: x.month)
df4_["month"] = df4_["month"].apply(lambda x: calendar.month_abbr[x].lower())
df4_["year"] = df4_['Date'].map(lambda x: x.year)
df4= df4_.iloc[::-1].reset_index(drop=True)

#display(df4.head())
df['employed'] = 0.0
df['unemployed'] = 0.0
df['unemployed_rate']=0.0

for i in range(len(df4)):    
    i_raw=df4.iloc[i,:]
    #print(i,i_raw.Value)
    df.loc[(df.month<=i_raw.month) & (df.year==i_raw.year.astype(float)),'employed']=i_raw.employed
    df.loc[(df.month<=i_raw.month) & (df.year==i_raw.year.astype(float)),'unemployed']=i_raw.unemployed
    df.loc[(df.month<=i_raw.month) & (df.year==i_raw.year.astype(float)),'unemployed_rate']=i_raw.unemployed_rate    

df.to_csv("after_append.csv",
          index=False,
          encoding="utf-8")

Sheet names are ['quaretrly', 'Metadata', 'monthly', 'daily']


In [30]:
# Ambiguous number of entries per feature:
# 288 unknowsn in job, 1857 unknowsn in education, 13020 unkowns in contact , 1840 other in poutcome, 36959 unkowns in poutcome

# poutcome 
ct_table_ind=pd.crosstab(df["poutcome"],df["pdays"]==999)
print('contingency_table between poutcome and pdays==999:\n{}'.format(ct_table_ind))
#change unknown to nonexistant where pdays==999, ow delete row
df.loc[((df['poutcome']=='unknown') & (df["pdays"] == 999)), "poutcome"] = 'non_existant'
indices_to_drop=df.loc[(df['poutcome']=="unknown") & (df['pdays']!=999)].index
df.drop(indices_to_drop,axis=0,inplace=True)
df.reset_index(inplace=True)
#change pdays=999 to pdays=0 to avoid contributing to outliers 
df.loc[df["pdays"] == 999, "pdays"] = 0
df.drop('index',axis=1,inplace=True)
#df

contingency_table between poutcome and pdays==999:
pdays     False   True
poutcome              
failure    4901      0
other      1840      0
success    1511      0
unknown       5  36954


In [31]:


df.loc[df.y=="yes",'y_equiv']=int(1)
df.loc[df.y=="no",'y_equiv']=int(0)

#flags for "unknown values" in https://machinelearningmastery.com/binary-flags-for-missing-values-for-machine-learning/

df['contact_missing'] = 0
df['poutcome_missing'] = 0
df['job_missing'] = 0
df['education_missing'] = 0

df.loc[df.contact=="unknown",'contact_missing']=1
df.loc[df.poutcome=="other",'poutcome_missing']=1
df.loc[df.job=="unknown",'job_missing']=1
df.loc[df.education=="unknown",'education_missing']=1

#nominal and cardinal 
#https://pianalytix.com/how-do-you-handle-missing-values-categorical-data-and-feature-scaling-in-machine-learning/
#sefidian.com/2021/07/02/measure-the-correlation-between-numerical-and-categorical-variables-and-the-correlation-between-two-categorical-variables-in-python-chi-square-and-anova/    


In [32]:
####################################################### balance 
df.loc[df["balance"]<0,'overdraft']=1 #this indicates bad credit score so overall not the best candidate to waste time on
df.loc[df["balance"]>=0,'overdraft']=0

df.loc[df["balance"]<0,'balance']=0 #as if balancerepresent +ve balance only
#binning of balance then label_encode
#assume5bgroups
bins_it=(df.balance.max()-df.balance.min())/5
bins=[]#[df.balance.min()]
for i in range(1,5):
    bins.append(bins_it*i+1)

bins.append(df.balance.max())
#bins=[bins_middle,df.balance.max()]
print('bins based on train data',bins)
orig_balance=df['balance']
df['balance'] = pd.cut(df['balance'], bins=5, labels=['vlow','low','med','high','vhigh'])


bins based on train data [20426.4, 40851.8, 61277.200000000004, 81702.6, 102127]


In [33]:
#make all categoric features numeric ! 
#cats
#job --> hot encoding, marital as well and education as well ? 
#default,housing,load,contact yes=1,no=0, contact=unknown==0(i have another repreesntative)
#month -> numerical
#day_of_week -> 
#fix ranges where numeric variables exist! 

binary=['default','housing','loan','contact','poutcome']
orig_binary=df[binary]

df.loc[df['default']=="yes",'default']=1
df.loc[df['default']=="no",'default']=0

df.loc[df['housing']=="yes",'housing']=1
df.loc[df['housing']=="no",'housing']=0

df.loc[df['loan']=="yes",'loan']=1
df.loc[df['loan']=="no",'loan']=0

df.loc[df['contact']=="cellular",'contact']=1
df.loc[(df['contact'] == "telephone") | (df['contact'] == "unknown"),'contact']=0 #we can cast unknowns as 0 now that we have a variable to indicate it ! 

df.loc[df['poutcome'] =="success",'poutcome']=1
df.loc[(df['poutcome'] == "non_existant") | (df['poutcome'] == "failure") | (df['poutcome'] == "other"),'poutcome']=0


months = {month: index for index, month in enumerate(calendar.month_abbr) if month}
months['Jan']
for month in months:
    #print(month.lower(),months[month])
    df.loc[df['month']==month.lower(),'month']=months[month]

df.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,...,consum_conf_ind,employed,unemployed,unemployed_rate,y_equiv,contact_missing,poutcome_missing,job_missing,education_missing,overdraft
0,58,management,married,tertiary,0,vlow,1,0,0,5,...,-24.4,4827.7,486.8,9.5,0.0,1,0,0,0,0.0
1,44,technician,single,secondary,0,vlow,1,0,0,5,...,-24.4,4827.7,486.8,9.5,0.0,1,0,0,0,0.0
2,33,entrepreneur,married,secondary,0,vlow,1,1,0,5,...,-24.4,4827.7,486.8,9.5,0.0,1,0,0,0,0.0
3,47,blue-collar,married,unknown,0,vlow,1,0,0,5,...,-24.4,4827.7,486.8,9.5,0.0,1,0,0,1,0.0
4,33,unknown,single,unknown,0,vlow,0,0,0,5,...,-24.4,4827.7,486.8,9.5,0.0,1,0,1,1,0.0


In [34]:
df.to_csv("before_hotencoding.csv",
          index=False,
          encoding="utf-8")

df['balance']=df.balance.cat.codes

In [35]:
#for high cardinality job according to how frequent a value appears, encode it .. BEFORE THIS STEP, SPLIT DATA FOR TRAINING AND TESTING AND ONLY USE TARGETS OF TRAINING DATA
X_train, X_test, y_train, y_test = train_test_split(df[df.columns[df.columns!='y_equiv']], df['y_equiv'], test_size=0.2, random_state=11)

#https://analyticsindiamag.com/a-complete-guide-to-categorical-data-encoding/
#https://towardsdatascience.com/dealing-with-categorical-variables-by-using-target-encoder-a0f1733a4c69
categories = X_train.job.unique()#df['job'].unique()
targets = y_train.astype('int32').unique()#df['y_equiv'].astype('int32').unique()
cat_list = []
for cat in categories:
    aux_dict = {}
    aux_dict['category'] = cat
    aux_df = X_train[X_train['job'] == cat]
    counts = df.loc[X_train[X_train['job'] == cat].index,'y_equiv'].value_counts()
    aux_dict['count'] = sum(counts)
    for t in targets:
# #        print(t)
        aux_dict['target_' + str(t)] = counts[t]
    cat_list.append(aux_dict)
cat_list = pd.DataFrame(cat_list)
cat_list['genre_encoded_dumb'] = cat_list['target_1'] / cat_list['count']
display(cat_list)
orig_job=cat_list

for cat in cat_list.category:
    educ_equiv=cat_list.loc[cat_list['category']==cat,'genre_encoded_dumb'].tolist()[0]
    #print(cat,educ_equiv)
    df.loc[df['job']==cat,'job']=educ_equiv#.astype('float64')

cat_list.to_csv("education_list.csv",
          index=False,
          encoding="utf-8")

,category,count,target_0,target_1,genre_encoded_dumb
0,blue-collar,7771,7201,570,0.073350
1,technician,6072,5392,680,0.111989
2,management,7588,6548,1040,0.137059
3,unemployed,1067,899,168,0.157451
4,retired,1831,1409,422,0.230475
5,entrepreneur,1182,1083,99,0.083756
6,admin.,4152,3643,509,0.122592
7,housemaid,986,907,79,0.080122
8,services,3272,2976,296,0.090465
9,self-employed,1280,1125,155,0.121094


In [36]:
# #ordinal encoding for education
orig_education=df['education']

ordinal_educ=df.education.unique().tolist()
ordinal_educ_encoded=[3,2,0,1]
display(ordinal_educ)
 
#for col in ["education"]:
#    df[col]=LabelEncoder().fit_transform(df[col])
#    print(LabelEncoder().fit(df[col]).classes_)
#label encoder assigns ['tertiary', 'secondary', 'unknown', 'primary'] as [0:3] hence losing the ordinal idea 
ind=0
for e in ordinal_educ:
    df.loc[df['education']==e,'education']=ordinal_educ_encoded[ind]
    ind=ind+1


['tertiary', 'secondary', 'unknown', 'primary']

In [37]:
orig_mairal=df.marital
dum_df = pd.get_dummies(df['marital'], columns=["marital"] )
# merge with main df bridge_df on key values
df = df.join(dum_df)
df.drop(['marital'],axis=1,inplace=True)

df.to_csv("before_hotencoding2.csv",
          index=False,
          encoding="utf-8")

In [38]:

    
#nominal data hot encoding
#for col in ["day_of_week","marital"]:
orig_dates=df[["day_of_week","year","month"]]
for col in ["day_of_week","year","month"]:    
    #str(col+'_encoded')
    days_enc=pd.DataFrame(OneHotEncoder().fit_transform(df[[col]]).toarray())
    colname=[]
    for day in range(len(days_enc.columns)):#.column:
        #print(day)
        colname.append(str(col)+str(day+1)) 
    
    #display(colname)
    df[colname] = days_enc


df.drop(['day_of_week','year','month'],axis=1,inplace=True)




D:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before

In [39]:
df.head(10)
df.to_csv("week9.csv",
          index=False,
          encoding="utf-8")

In [40]:
# X_train, X_test, y_train, y_test


# #binning of balance then label_encode
# #assume5bgroups
# bins_it=(X_train.balance.max()-X_train.balance.min())/5
# bins=[]#[df.balance.min()]
# for i in range(1,5):
#     bins.append(bins_it*i+1)
# bins.append(X_train.balance.max())
# #bins=[bins_middle,df.balance.max()]
# print(bins)
# #orig_balance=df['balance']
# #df['balance'] = pd.cut(df['balance'], bins=5, labels=['vlow','low','med','high','vhigh'])


# #binning of balance then label_encode
# #assume5bgroups
# bins_it=(df.balance.max()-df.balance.min())/5
# bins=[]#[df.balance.min()]
# for i in range(1,5):
#     bins.append(bins_it*i+1)
# bins.append(df.balance.max())
# #bins=[bins_middle,df.balance.max()]
# print(bins)
# #orig_balance=df['balance']
# #df['balance'] = pd.cut(df['balance'], bins=5, labels=['vlow','low','med','high','vhigh'])
# X_train.describe()



# ####################################################### balance 


# df.loc[df["balance"]<0,'balance']=0 #as if balancerepresent +ve balance only
# #binning of balance then label_encode
# #assume5bgroups
# bins_it=(df.balance.max()-df.balance.min())/5
# bins=[]#[df.balance.min()]
# for i in range(1,5):
#     bins.append(bins_it*i+1)

# bins.append(df.balance.max())
# #bins=[bins_middle,df.balance.max()]
# bins
# orig_balance=df['balance']
# df['balance'] = pd.cut(df['balance'], bins=5, labels=['vlow','low','med','high','vhigh'])